<a href="https://colab.research.google.com/github/kaisacht/Project1/blob/main/FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
 
from tensorflow import keras
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 20_day_TRAIN.txt to 20_day_TRAIN.txt


In [ ]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y

In [ ]:
x_train, y_train = readucr('20_day_TRAIN.txt')
x_test, y_test = readucr('20_day_TEST.txt')

In [ ]:
print(x_test,y_test)

[[ 29.17  29.03  28.87 ...  29.81  29.83  29.4 ]
 [ 29.44  29.45  29.53 ...  30.14  30.35  30.37]
 [ 30.13  29.41  29.22 ...  27.92  27.73  27.76]
 ...
 [275.84 272.38 276.61 ... 275.11 274.52 267.98]
 [263.85 260.66 261.33 ... 237.57 235.2  236.9 ]
 [234.5  234.73 234.41 ... 234.87 234.5  241.3 ]] [3. 3. 1. 3. 3. 3. 1. 3. 1. 3. 3. 3. 3. 1. 1. 1. 3. 3. 1. 1. 3. 3. 1. 1.
 1. 1. 1. 1. 3. 3. 3. 3. 3. 1. 3. 3. 3. 3. 3. 1. 3. 3. 3. 1. 1. 3. 1. 3.
 3. 3. 3. 3. 1. 1. 3. 1. 1. 3. 1. 3. 3. 1. 1. 3. 3. 3. 1. 1. 1. 3. 1. 3.
 3. 1. 1. 3. 1. 3. 3. 3. 3. 3. 1. 1. 1. 3. 3. 3. 1. 1. 3. 3. 3. 1. 3. 3.
 3. 3. 1. 3. 1. 1. 1. 1. 3. 3. 1. 1. 3. 1. 3. 3. 3. 3. 1. 1. 3. 3. 1. 1.
 3. 3. 1. 3. 3. 1. 3. 3. 3. 3. 3. 3. 1. 3. 1. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 1. 3. 1. 3. 1. 3. 3. 3. 3. 1. 3. 3. 1. 3. 3. 3. 3. 3. 3. 1. 3.
 3. 3. 3. 3. 3. 1. 1. 3. 3. 3. 1. 3. 3. 1. 3. 3. 3. 3. 1. 3. 3. 1. 1. 1.
 3. 1. 1. 1. 3. 1. 1. 3.]


In [ ]:
nb_epochs = 2000

In [ ]:
flist  = ['Adiac']
for each in flist:
    fname = each
    x_train, y_train = readucr('20_day_TRAIN.txt')
    x_test, y_test = readucr('20_day_TEST.txt')
    nb_classes = len(np.unique(y_test))
    batch_size = min(x_train.shape[0]/10, 16)
    
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    
    
    Y_train = keras.utils.to_categorical(y_train, nb_classes)
    Y_test = keras.utils.to_categorical(y_test, nb_classes)
    
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
    x_test = (x_test - x_train_mean)/(x_train_std)
    x_train = x_train.reshape(x_train.shape + (1,1,))
    x_test = x_test.reshape(x_test.shape + (1,1,))

    x = keras.layers.Input(x_train.shape[1:])
#    drop_out = Dropout(0.2)(x)
    conv1 = keras.layers.Conv2D(128, 8, 1, padding='same')(x)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation('relu')(conv1)
    
#    drop_out = Dropout(0.2)(conv1)
    conv2 = keras.layers.Conv2D(256, 5, 1, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    
#    drop_out = Dropout(0.2)(conv2)
    conv3 = keras.layers.Conv2D(128, 3, 1, padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    
    full = keras.layers.GlobalAveragePooling2D()(conv3)
    out = keras.layers.Dense(nb_classes, activation='softmax')(full)
    
    
    model = keras.models.Model(inputs=x, outputs=out)
     
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
    hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
    #Print the testing results which has the lowest training loss.
    log = pd.DataFrame(hist.history)

Epoch 1/2000
13/13 [==============================] - 2s 123ms/step - loss: 0.8078 - accuracy: 0.5792 - val_loss: 0.6966 - val_accuracy: 0.4050 - lr: 0.0010
Epoch 2/2000
13/13 [==============================] - 1s 108ms/step - loss: 0.7107 - accuracy: 0.5792 - val_loss: 0.7664 - val_accuracy: 0.3700 - lr: 0.0010
Epoch 3/2000
13/13 [==============================] - 1s 108ms/step - loss: 0.6764 - accuracy: 0.5842 - val_loss: 1.0089 - val_accuracy: 0.3850 - lr: 0.0010
Epoch 4/2000
13/13 [==============================] - 1s 107ms/step - loss: 0.6876 - accuracy: 0.5495 - val_loss: 0.7404 - val_accuracy: 0.4600 - lr: 0.0010
Epoch 5/2000
13/13 [==============================] - 1s 108ms/step - loss: 0.6167 - accuracy: 0.7129 - val_loss: 0.7860 - val_accuracy: 0.4500 - lr: 0.0010
Epoch 6/2000
13/13 [==============================] - 1s 106ms/step - loss: 0.6309 - accuracy: 0.6584 - val_loss: 0.6717 - val_accuracy: 0.6300 - lr: 0.0010
Epoch 7/2000
13/13 [==============================] - 1s 1

In [ ]:
    log = pd.DataFrame(hist.history)
    print(log)

          loss  accuracy   val_loss  val_accuracy      lr
0     0.807764  0.579208   0.696636         0.405  0.0010
1     0.710733  0.579208   0.766435         0.370  0.0010
2     0.676439  0.584158   1.008912         0.385  0.0010
3     0.687641  0.549505   0.740442         0.460  0.0010
4     0.616656  0.712871   0.785953         0.450  0.0010
...        ...       ...        ...           ...     ...
1995  0.111602  0.950495   4.498240         0.750  0.0001
1996  0.147313  0.960396   5.534539         0.750  0.0001
1997  0.127579  0.955446   3.975681         0.755  0.0001
1998  0.161190  0.935644   5.099497         0.715  0.0001
1999  0.095549  0.960396  11.859540         0.685  0.0001

[2000 rows x 5 columns]


In [ ]:
log_np = np.array(log)
log_np
min_index = (np.argmin(log_np, axis=0))
print(log_np[min_index[0]])

[7.78331012e-02 9.75247502e-01 1.75659955e+00 8.14999998e-01
 9.99999975e-05]


In [ ]:
log_np = np.array(log)
log_np
max_index = (np.argmax(log_np, axis=0))
print(log_np[max_index[3]])

[2.36705571e-01 9.00990129e-01 2.13789582e-01 9.44999993e-01
 2.50000012e-04]
